In [ ]:
 !pip install hdm2 --quiet

# I. Hallucination Detection for exemple

Mô tả:  Đầu vào gồm 
{rcontext , promt , reponse}

In [ ]:

from hdm2 import HallucinationDetectionModel
hdm_model = HallucinationDetectionModel()


In [ ]:
import pandas as pd

# Đọc file
df = pd.read_csv("/kaggle/input/hcmus-fourgight-public-test/vihallu-public-test.csv")



# Reset lại index
df = df.reset_index(drop=True)

print(df.head())


In [ ]:
datatest = df

In [ ]:
import re

# Hàm làm sạch text
def clean_text(x):
    if not isinstance(x, str):
        return ""
    x = x.strip()
    x = re.sub(r"\s+", " ", x)
    return x

# Tiền xử lý DataFrame
for col in ["prompt", "context", "response"]:
    datatest[col] = datatest[col].map(clean_text)

# Chạy từng dòng với for loop
results_list = []
labels_list = []

for idx, row in datatest.iterrows():
    prompt   = str(row.get("prompt", ""))
    context  = "\n" + str(row.get("context", ""))+ "\n"
    response = "\n" + str(row.get("response", ""))+ "\n"


    results = hdm_model.apply(prompt, context, response)
    score = results['adjusted_hallucination_severity']

    results_list.append(score)
    labels_list.append(0 if score < 0.4 else 1)  # 0 = không hallucination, 1 = hallucination

# Lưu lại kết quả vào DataFrame
datatest["results"] = results_list
datatest["label_prep"] = labels_list

# Xem kết quả
print(datatest[["id", "results", "label_prep"]].head())


import pandas as pd
from sklearn.metrics import accuracy_score


label_mapping = {"no": 0, "extrinsic": 1, "intrinsic": 1}
datatest["labelmap"] = datatest["label"].map(label_mapping)



# Tính accuracy
accuracy = accuracy_score(datatest["label_prep"], datatest["labelmap"])

print("Accuracy:", accuracy)

In [ ]:
datatest

In [ ]:
datatest.to_csv("result.csv")